<a href="https://colab.research.google.com/github/Wesley-Janson/transformers_for_human_vs_ai_text_identification/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Import relevant packages and data_loader.py
#import data_loader

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d, MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split

In [26]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd

In [35]:
from google.colab import files
 
 
uploaded = files.upload()

Saving data.csv to data (2).csv


In [36]:
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)
  print(len(df))
  
  text = df['intro'].values
  labels = df['type'].values
  return labels,text


In [37]:

# This function processes training data, establishing number IDs for each vocabulary word,
# converting word sequence into ID sequence (input_as_ids), and providing dict
# to map from word to its ID (word2id), and list to map from ID back to word (id2word)
def process_training_data(corpus_text):
        """Tokenizes a text file."""
        # Create the model's vocabulary and map to unique indices
        word2id = {}
        id2word = []
        list_of_inputs = []
        for entry in corpus_text:
            for i,word in enumerate(entry):
                if 7<i<=30:
                    if word not in word2id:
                        id2word.append(word)
                        word2id[word] = len(id2word) - 1

            # Convert string of text into string of IDs in a tensor for input to model
            input_as_ids = []
            for i,word in enumerate(entry):
                if 7<i<=30:
                    input_as_ids.append(word2id[word])
            if len(input_as_ids) == 23:
              list_of_inputs.append(input_as_ids)
            # final_ids = torch.LongTensor(input_as_ids)
        list_of_inputs = torch.Tensor(list_of_inputs)

        return list_of_inputs,word2id,id2word

In [38]:
!pwd
!ls

/content
'data (1).csv'	'data (2).csv'	 data.csv  'data.csv '	 drive	 sample_data


In [39]:
# Run data loader
labels,text = load_data('/content/data.csv')
print(len(text))
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)
tokenizer = get_tokenizer("basic_english")
list_of_tokens_train = [tokenizer(x) for x in train_x]
list_of_tokens_val = [tokenizer(x) for x in val_x]

train_x,word2id,id2word = process_training_data(list_of_tokens_train)

val_x,word2id,id2word = process_training_data(list_of_tokens_val)
print(train_x)
print(type(train_x))
train_y = torch.Tensor(train_y)
val_y = torch.Tensor(val_y)
#############################
#### SPLIT INTO TRAIN/TEST HERE??????
#############################

0
0


ValueError: ignored

In [ ]:
print(train_x.shape)
print(train_y.shape

NameError: ignored

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
labels,text = load_data('data.csv')
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)
print(train_x)
print(type(train_x))
print(train_x.shape)

['The Harmonie Club is a private social club in Bern, Switzerland, that was founded in 1884. It is one of the oldest and most prestigious clubs in Switzerland.\n\nThe Harmonie Club was founded on October 13, 1884, by a group of friends who were looking for a place to socialize and enjoy their music. The club originally met at the home of its first president, Dr. Julius Hutter. Today, the club is located on Bellevuegasse in Bern.\n\nSince its inception, the Harmonie Club has been committed to promoting music and the arts. It has hosted many famous musicians, including Joseph Joachim, Wilhelm Furtwängler, and Arturo Toscanini. The club also sponsors a number of performances and festivals throughout the year.\n\nThe Harmonie Club is open to all members of the Swiss community who are interested in music and the arts. Membership is free of charge.'
 "Red Sea Project is a land and property development announced by the Saudi Crown Prince Mohammad bin Salman in July 2017. It is planned to be e

In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    print(label_list)
    print(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

loader = torch.utils.dataloader(train, 
                                batch_size=1, # choose your batch size
                                shuffle=True)
dataloader = DataLoader(pd.read_csv('data.csv'), batch_size=8, shuffle=False, collate_fn=collate_batch)

In [ ]:
# Define Architecture

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv1d(59704, 4000, kernel_size=3, stride=1, padding=1),
            BatchNorm1d(23),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv1d(4000, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm1d(23),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(59704, 4000, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(4000, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=10, bias=True)
  )
)


In [ ]:
# Create train function
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    print(train_x)
    print(train_y)
    print(train_x.shape)
    print(train_y.shape)
    x_train, y_train = Variable(torch.Tensor(train_x)), Variable(torch.Tensor(train_y))
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
  train(epoch)
print(train_losses)
print(val_loses)

tensor([[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 1.5000e+01, 1.6000e+01,
         1.0000e+01],
        [3.0000e+00, 1.0000e+01, 1.1000e+01,  ..., 3.1000e+01, 3.2000e+01,
         2.1000e+01],
        [1.9000e+01, 3.3000e+01, 1.4000e+01,  ..., 3.3000e+01, 1.4000e+01,
         4.3000e+01],
        ...,
        [7.7557e+04, 5.1000e+01, 7.7558e+04,  ..., 1.9000e+01, 7.7370e+03,
         8.6000e+01],
        [1.2271e+04, 3.0000e+00, 3.6000e+01,  ..., 4.4520e+03, 1.4000e+01,
         8.2000e+01],
        [0.0000e+00, 1.1000e+01, 8.3900e+02,  ..., 1.9708e+04, 3.2000e+01,
         9.0900e+02]])
tensor([1., 1., 0.,  ..., 0., 0., 1.])
torch.Size([59704, 23])
torch.Size([60000])
